In [4]:
from influxdb import InfluxDBClient
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


class DateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        None
    
    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        X = X.reset_index()
        X['datetime'] = X['time'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
        X['year'] = X['datetime'].apply(lambda x: x.year)
        X['month'] = X['datetime'].apply(lambda x: x.month)
        X['day'] = X['datetime'].apply(lambda x: x.day)
        X['hour'] = X['datetime'].apply(lambda x: x.hour)
        X['minute'] = X['datetime'].apply(lambda x: x.minute)
        X['day_of_week'] = X['datetime'].apply(lambda x: x.dayofweek)
        y = X['Total']
        return X[['year', 'month', 'day', 'hour', 'minute', 'day_of_week', 'Total']], y

class Shifter(BaseEstimator, TransformerMixin):
    def __init__(self):
        None
    
    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None, hours = 3):
        X = X[0]
        for i in range(hours):
            colname = f"{i+1}hourback"
            X[colname] = X['Total'].shift(60 * (-i-1))
        X = X.dropna()
        y = X['Total']
        X = X.drop(columns = ['Total'], axis = 1)
        return X, y


if __name__ == '__main__':
    client = InfluxDBClient(host = 'influxus.itu.dk', port = 8086, username = 'lsda', password = 'icanonlyread')
    client.switch_database('orkney')

    results = client.query('SELECT * FROM "Generation" where time > now() - w ORDER BY time')
    points = results.get_points()
    values = results.raw['series'][0]["values"]
    columns = results.raw['series'][0]["columns"]
    generation_df = pd.DataFrame(values, columns = columns).set_index("time")

    preparation_pipeline = Pipeline([
        ('date_worker', DateTransformer()),
        ('shifter', Shifter())
    ])

    processed_data = preparation_pipeline.fit_transform(generation_df)

    dim1, dim2 = processed_data[0].shape

    X = processed_data[0]
    y = processed_data[1]

    lreg_model = Lasso().fit(processed_data[0].iloc[:int(0.8*dim1)], processed_data[1][:int(0.8 * dim1)])
    print(lreg_model.score(processed_data[0].loc[int(0.8 * dim1):], processed_data[1][int(0.8 * dim1):]))

0.9629396261899098


In [3]:
from sklearn.model_selection import TimeSeriesSplit


tscv = TimeSeriesSplit()

In [10]:
# models
from sklearn.svm import SVR

lr_model = LinearRegression()
lasso_model = Lasso()
ridge_model = Ridge()
rf_model = RandomForestRegressor()
svm_model = SVR()

In [14]:
def cv_model(model, X, y):
    for train_index, test_index in tscv.split(X):
        model.fit(X.iloc[train_index], y[train_index])
        print(f"R2 score:  {model.score(X.iloc[test_index], y[test_index])}")
        predictions = model.predict(X.iloc[test_index])
        print(f"MAE: {mean_absolute_error(predictions, y[test_index])}")

In [15]:
for model in [lr_model, lasso_model, ridge_model, rf_model, svm_model]:
    print("\n\n")
    cv_model(model, X, y)

R2 score:  0.8604324292323158
MAE: 2.715149728553486
R2 score:  0.9214147519755347
MAE: 2.7492130656904066
R2 score:  0.8402926007421173
MAE: 3.272478335783512
R2 score:  0.9437317595183369
MAE: 2.148732121852667
R2 score:  0.910001174955952
MAE: 1.686147468020713
